# The Paleobiology Database Tutorial
This tutorial will teach you how to access data from the [Paleobiology Database (PBDB)](https://paleobiodb.org/navigator/).

## What is the Paleobiology Database?
PBDB is a public database of paleontological data that anyone can use, maintained by an international non-governmental group of paleontologists. One of its main features is its navigator, which allows a user to sort data by geological time, taxa, authorizer, stratigraphy, and more. PBDB is run by the Department of Geoscience at the University of Wisconsin-Madison. The project team consists of Shanan Peters, Michael McClennan, and John Czaplewski. 

## How do you access the data?
PBDB is free to use and has no requirements for access. After sorting through the [PBDB navigator](https://paleobiodb.org/navigator/) and finding the dataset you want to download, click on the button to the left called "save map data". A window will appear, giving you two choices. You can either download the data as a CSV, JSON, TSV, or RIS file, or you can obtain a URL that can be used for external scripts such as R or Python. If you choose to download the data as a file, it can be used automatically for analysis. However, accessing the data by making HTTP requests is a little more intensive. This tutorial will teach you how to obtain the data desired by using the URL, and will require installation of Python and Jupyter. Instructions for download can be found [here for Python](https://realpython.com/installing-python/) and [here for Jupyter](http://jupyter.org/install). In addition, documentation for the data service (including data recorded in the file and instructions on usage) can be found [here](https://paleobiodb.org/data1.2).


## Obtaining data through HTTP requests

1) Obtain your URL by clicking on the "Get URL" tab in the window discussed above. <br>
2) Click "Get URL" for the label "Data URL". Copy this url and open a new Jupyter notebook. <br>
3) In this example, I will request a URL corresponding to a dataset on jurassic dinosaurs fossils in the western US.

In [89]:
import requests
import csv
import json
URL = "https://paleobiodb.org/data1.2/occs/list.json?lngmin=-143.2617&lngmax=-75.7617&latmin=31.3536&latmax=48.5748&interval_id=15&base_id=10707,52775&show=coords,attr,loc,prot,time,strat,stratext,lith,lithext,geo,rem,ent,entname,crmod&datainfo"

r = requests.get(url=URL) #store the data obtained through the HTTP request in r
data = r.json() #converts the data in r to a json file
with open("dino_data.json", "w") as write_file:
    json.dump(data, write_file) #creates file dino_data.json

If you want to examine the raw dino_data.json file, you can find it in the jupyter folder tree. Now, let's convert this json file into a workable pandas dataframe.

In [106]:
import pandas as pd
from pandas.io.json import json_normalize

#We need to normalize the data by converting the structured json file into a flat table.
df = pd.DataFrame.from_dict(json_normalize(data))
df

,access_time,data_license,data_provider,data_source,data_url,documentation_url,elapsed_time,license_url,parameters.base_id,parameters.interval_id,parameters.latmax,parameters.latmin,parameters.lngmax,parameters.lngmin,parameters.show,parameters.taxon_status,parameters.timerule,records,title
0,Fri 2018-10-19 07:01:23 GMT,Creative Commons CC-BY,The Paleobiology Database,The Paleobiology Database,http://paleobiodb.org/data1.2/occs/list.json?l...,http://paleobiodb.org/data1.2/occs/list_doc.html,0.492,http://creativecommons.org/licenses/by/4.0/,"10707,52775",15,48.5748,31.3536,-75.7617,-143.2617,"coords,attr,loc,prot,time,strat,stratext,lith,...",all,major,"[{'oid': 'occ:139307', 'eid': 'rei:34164', 'ci...",PBDB Data Service


Scrolling through this flat dataframe, we notice that the column "records" contains a nested json list. Because the other columns of this dataframe aren't very useful to us, we can hone in on the "records" column by calling json_normalize on a second argument that will allow us to convert nested elements to dataframes.

In [166]:
dino_df = pd.DataFrame.from_dict(json_normalize(data, ["records"]))
dino_df.head()

,ath,ati,cc2,cid,cny,cxi,dcr,dmd,eag,eid,...,slb,slo,sls,smb,ssc,stp,tdf,tec,tid,tna
0,M. Carrano,prs:14,US,col:11924,Mesa,41,2017-11-02 14:56:21,2017-11-02 14:56:21,157.3,rei:34164,...,NaN,NaN,NaN,Brushy Basin,bed,Colorado,NaN,NaN,txn:52961,Camarasaurus grandis
1,M. Carrano,prs:14,US,col:11925,Mesa,125,2001-09-19 09:11:44,2005-11-22 09:28:42,157.3,NaN,...,NaN,NaN,NaN,Salt Wash,bed,Colorado,NaN,NaN,txn:54168,Camarasaurus supremus
2,M. Carrano,prs:14,US,col:13212,Albany,41,2002-01-29 12:56:29,2014-07-25 16:16:45,157.3,NaN,...,NaN,NaN,NaN,Brushy Basin,bed,Wyoming,subjective synonym of,NaN,txn:52962,Allosaurus fragilis
3,M. Carrano,prs:14,US,col:13214,Converse,41,2002-01-29 13:00:06,2014-08-04 09:06:19,157.3,NaN,...,NaN,NaN,NaN,NaN,group of beds,Wyoming,subjective synonym of,NaN,txn:52962,Allosaurus fragilis
4,M. Carrano,prs:14,US,col:13214,Converse,41,2002-01-29 13:00:06,2014-08-04 09:06:19,157.3,NaN,...,NaN,NaN,NaN,NaN,group of beds,Wyoming,NaN,NaN,txn:52962,Allosaurus fragilis


In [168]:
dino_df.shape

(1116, 57)

Now this looks like something we can work with! We see that this table has 1116 entries and 57 features in total, not a particularly large dataset given that we're working with dinosaur fossils in only the Western US. We can start cleaning our data now by renaming unintuitive column names, converting data values into something more workable (i.e. the time and date format of the "dcr" and "dmd" columns), and examining missing values in the sib, sio, and sis columns, to name a few. 

In [169]:
dino_df.rename({"ath": "authorizer",
               "ati": "authorizer_no",
               "cc2": "country",
               "cid": "collection_no",
               "cny": "county",
               "cxi": "cx_int_no",
               "dcr": "created",
                "dmd": "modified",
               "eag": "max_ma",
               "eid": "reid_no",
               "smb": "member",
               "ssc": "stratscale",
               "stp": "state",
               "tdf": "diference",
               "tid": "accepted_no",
                "slb": "local_bed",
                "slo": "local_order",
                "sls": "local_section",
               "tna": "identified_name"}, inplace=True, axis="columns")

#I have renamed the columns I can see, and it will be tedious work to rename the rest. I will leave it as is for now.
dino_df.head()

,authorizer,authorizer_no,country,collection_no,county,cx_int_no,created,modified,max_ma,reid_no,...,local_bed,local_order,local_section,member,stratscale,state,diference,tec,accepted_no,identified_name
0,M. Carrano,prs:14,US,col:11924,Mesa,41,2017-11-02 14:56:21,2017-11-02 14:56:21,157.3,rei:34164,...,NaN,NaN,NaN,Brushy Basin,bed,Colorado,NaN,NaN,txn:52961,Camarasaurus grandis
1,M. Carrano,prs:14,US,col:11925,Mesa,125,2001-09-19 09:11:44,2005-11-22 09:28:42,157.3,NaN,...,NaN,NaN,NaN,Salt Wash,bed,Colorado,NaN,NaN,txn:54168,Camarasaurus supremus
2,M. Carrano,prs:14,US,col:13212,Albany,41,2002-01-29 12:56:29,2014-07-25 16:16:45,157.3,NaN,...,NaN,NaN,NaN,Brushy Basin,bed,Wyoming,subjective synonym of,NaN,txn:52962,Allosaurus fragilis
3,M. Carrano,prs:14,US,col:13214,Converse,41,2002-01-29 13:00:06,2014-08-04 09:06:19,157.3,NaN,...,NaN,NaN,NaN,NaN,group of beds,Wyoming,subjective synonym of,NaN,txn:52962,Allosaurus fragilis
4,M. Carrano,prs:14,US,col:13214,Converse,41,2002-01-29 13:00:06,2014-08-04 09:06:19,157.3,NaN,...,NaN,NaN,NaN,NaN,group of beds,Wyoming,NaN,NaN,txn:52962,Allosaurus fragilis


I have renamed the columns I can see, and it will be tedious work to rename the rest. I will leave it as is for now. We can look at the column labels with the following command:

In [147]:
dino_df.columns

Index(['authorizer', 'authorizer_no', 'country', 'collection_no', 'county',
       'cx_int_no', 'created', 'modified', 'max_ma', 'reid_no', 'eni', 'ent',
       'env', 'ff1', 'ff2', 'flg', 'gcm', 'ggc', 'gsc', 'idn', 'idr', 'iid',
       'la1', 'la2', 'lag', 'lat', 'ldc', 'lf1', 'lf2', 'lm1', 'lm2', 'lng',
       'lt1', 'lt2', 'mdf', 'mdi', 'ocm', 'oei', 'oid', 'oli', 'prc', 'ptd',
       'rid', 'rnk', 'scm', 'sfm', 'sgr', 'local_bed', 'local_order',
       'local_section', 'member', 'stratscale', 'state', 'diference', 'tec',
       'accepted_no', 'identified_name'],
      dtype='object')

Let's pick out some columns that look interesting.

In [162]:
interesting = dino_df[["identified_name", "state", "lat", "lng", "env", "created"]]
interesting.head()

,identified_name,state,lat,lng,env,created
0,Camarasaurus grandis,Colorado,39.068802,-108.699989,fluvial-lacustrine indet.,2017-11-02 14:56:21
1,Camarasaurus supremus,Colorado,39.111668,-108.717499,fluvial-lacustrine indet.,2001-09-19 09:11:44
2,Allosaurus fragilis,Wyoming,41.893101,-106.041000,wet floodplain,2002-01-29 12:56:29
3,Allosaurus fragilis,Wyoming,42.035831,-106.389999,terrestrial indet.,2002-01-29 13:00:06
4,Allosaurus fragilis,Wyoming,42.035831,-106.389999,terrestrial indet.,2002-01-29 13:00:06


According to the [documentation of date/time](https://paleobiodb.org/data1.2/general/datetime_doc.html), the "created" column is not the day the fossil was found, but rather the day and time the database entry was created. Disappointing, but it can still be used as a rough estimate of the time of discovery. Let's peruse our data for anything interesting:

In [163]:
dino_df["county"].unique() 

array(['Mesa', 'Albany', 'Converse', 'Coconino', nan, 'Jefferson',
       'Hot Springs', 'Hot Spings', 'Uintah', 'Bernalillo', 'San Juan',
       'Fremont', 'Cibola', 'Fall River', 'Carbon', 'Union', 'Sandoval',
       'Harding', 'Rio Arriba', 'Gunnison', 'Navajo', 'Montrose',
       'Big Horn', 'Emery', 'Natrona', 'Johnson', 'Cimarron', 'Grand',
       'Park', 'Meade', 'Lawrence', 'Crook', 'Mohave', 'Washington',
       'San Bernardino', 'Otero', 'Las Animas', 'Delta', 'Teton', 'Kane',
       'Daggett', 'Summit', 'Garfield', 'Weston', 'Wayne', 'Juab',
       'Larimer', 'McKinley', 'Apache', 'Loudoun', 'Niobrara', 'Washakie',
       'Moffat', 'San Miguel', 'Eagle', 'Pitkin', 'Montezuma', 'Utah',
       'El Paso', 'Guadalupe', 'Socorro', 'Quay', 'Bear Lake'],
      dtype=object)

In [164]:
dino_df["state"].unique()

array(['Colorado', 'Wyoming', 'Arizona', 'Utah', 'New Mexico',
       'South Dakota', 'Montana', 'Oklahoma', 'California',
       'Arizona/Utah', 'Virginia', 'Idaho'], dtype=object)

There seems to be a value for Arizona/Utah! Let's examine that.

In [165]:
interesting[interesting["state"] == "Arizona/Utah"]

,identified_name,state,lat,lng,env,created
620,Navahopus coyoteensis,Arizona/Utah,36.999443,-112.00972,dune,2008-08-13 10:39:11
621,Grallator (Eubrontes),Arizona/Utah,36.999443,-112.00972,dune,2006-05-15 12:07:03
622,Otozoum,Arizona/Utah,36.999443,-112.00972,dune,2006-05-15 12:07:03
724,Theropoda,Arizona/Utah,36.999443,-112.00972,dune,2008-08-13 10:30:14


It seems all these specimen were found in the same area [(shown here)](https://www.google.com/maps/place/36%C2%B059'58.0%22N+112%C2%B000'35.0%22W/@37.0015359,-112.0181786,3068m/data=!3m1!1e3!4m5!3m4!1s0x0:0x0!8m2!3d36.999443!4d-112.00972), potentially on two separate days given the day of the entry creations. [Theropoda](https://en.wikipedia.org/wiki/Theropoda) is a dinosaur suborder characterized for their three toes and [Grallator](https://en.wikipedia.org/wiki/Grallator) is the fossilized footprint of theropods with similar characterizations with [Navahopus](http://fossilworks.org/bridge.pl?a=taxonInfo&taxon_no=131329). On the other hand, [Otozoum](https://en.wikipedia.org/wiki/Otozoum) is the fossilized footprint of a four-toed dinosaur, suggesting a variety of dinosaur activity in the same area.

In [177]:
interesting["env"].unique()

array(['fluvial-lacustrine indet.', 'wet floodplain',
       'terrestrial indet.', '"channel"', 'pond', 'coarse channel fill',
       'crevasse splay', 'dune', '"floodplain"', 'fluvial indet.',
       'levee', 'interdune', 'eolian indet.', 'fine channel fill',
       'dry floodplain', 'lacustrine indet.', 'coastal indet.',
       'marine indet.', 'channel lag', 'marginal marine indet.',
       'peritidal', 'estuary/bay', 'lacustrine - large',
       'carbonate indet.', 'deltaic indet.', 'delta plain',
       'lacustrine delta front'], dtype=object)

It seems that most of these environments are or were bodies of water. This makes sense, since the lack of oxygen in watery graves helps preserve fossil remains.